In [1]:
import pandas as pd 
import numpy as np
import os 

In [2]:
# analysis folder version  
analysis_version = '007'

# video metric folder version 
version = '006'

# Brainwalk data path

In [3]:
bw_path = r'C:\Users\mmccu\AppData\Local\Temp\ccsecure\2025_04_17_BrainWalk_AllData_Long_MM.xlsx'
path, bw_filename = os.path.split(bw_path)
bw_filename, ext = os.path.splitext(bw_filename)
bw_filename

'2025_04_17_BrainWalk_AllData_Long_MM'

## Define Function 

In [4]:
# create ordinal value of EDSS severity 
# 0-2 (mild), 2.5-4 (moderate), 4.5+ (severe)

# Function to categorize EDSS severity
def categorize_edss(edss_value):
    if 0 <= edss_value <= 2:
        return 1, 'mild'
    elif 2.5 <= edss_value <= 4:
        return 2, 'moderate'
    elif edss_value >= 4.5:
        return 3, 'severe'
    else:
        return None, None  # Handle cases outside the defined ranges

In [5]:
# create ordinal value for T25FW 
def categorize_t25fw(t25fw_value):
    if 0 <= t25fw_value < 6:
        return 1, 'under_6'
    elif 6 <= t25fw_value <= 7.99:
        return 2, '6_to_8'
    elif t25fw_value >= 8:
        return 3, 'over_8'
    else:
        return None, None  # Handle cases outside the defined ranges

In [6]:
# task = gait_vertical_PWS_1 or gait_vertical_FW_1
def merge_bw_zv(bw_df, zv_df, task, out_path):

    # filter zv to only include one task (bw drop columns, zv drop rows) 
    zv_task_df = zv_df[zv_df['task_pose_zv'] == task]
    print('confirm all one task')
    print(pd.unique(zv_task_df['task_pose_zv']))

    print('total zeno videos') 
    print(len(zv_task_df))
        
    # filter bw ids dataset to only those included in video data set 
    zv_in_bw_df = bw_df[bw_df['bw_id'].isin(zv_task_df['id_video'])]
    zv_in_bw_df.to_csv(os.path.join(out_path, 'zv_id_in_bw_df_' + task + '.csv')) # save excel 

    print('total bw rows with id in video dataset') 
    print(len(zv_in_bw_df))

    # merge bw data set rows with zeno videos rows 
        # merge bw data set rows with zeno videos rows 
        # id and date needs to be the same 
        # should only use each brainwalk visit once - once PWS_1 video per person 

    merged_bw_zv = []

    # Loop through each row in zv_task_df
    for index, zv_row in zv_task_df.iterrows():
       
        current_id = zv_row['id_video']
        current_date = zv_row['visit_date_video']
        zv_row_df = pd.DataFrame([zv_row])

        # Find rows in brainwalk data set with same id and same date as current zv data 
        zv_in_bw_current_id_rows = zv_in_bw_df[(zv_in_bw_df['bw_id'] == current_id) & (zv_in_bw_df['trialdate'] == current_date)]
        #zv_in_bw_current_id_date_rows = zv_in_bw_current_id_rows[zv_in_bw_current_id_rows['visit_date'] == current_date]
   
        if len(zv_in_bw_current_id_rows) == 1: 
            bw_row_to_merge = zv_in_bw_current_id_rows
            # merge bw and zv, store merged row
            merged_row = zv_row_df.merge(bw_row_to_merge, left_on='id_video', right_on='bw_id')
            merged_bw_zv.append(merged_row)
        
        # if more than one row for the id and date, pick one with least na values 
        elif len(zv_in_bw_current_id_rows) > 1:
            bw_row_to_merge = zv_in_bw_current_id_rows.loc[[zv_in_bw_current_id_rows.isna().sum(axis=1).idxmin()]]
            # merge bw and zv, store merged row
            merged_row = zv_row_df.merge(bw_row_to_merge, left_on='id_video', right_on='bw_id')
            merged_bw_zv.append(merged_row)

            print('multiple rows for the id and date combo')
            print(current_id)
            print(current_date)

        else: 
            print('No matching id and daterow from video vs mat')
            print(current_id)
            print(current_date)


    # merge all bw and zv data together 
    merged_bw_zv_df = pd.concat(merged_bw_zv)
    merged_bw_zv_df = merged_bw_zv_df.reset_index(drop=True) # reset index 

    # check same ID for each row 
    print('mismatched zeno video vs brainwalk id')
    print(sum(merged_bw_zv_df['id_video'] != merged_bw_zv_df['bw_id']))

    print('mismatched zeno video vs brainwalk date')
    print(sum(merged_bw_zv_df['visit_date_video'] != merged_bw_zv_df['trialdate']))

    # saved merged df for future reference 
    merged_bw_zv_df.to_csv(os.path.join(out_path,  'zv_bw_merged_' + task + '_raw.csv'))

    return merged_bw_zv_df

In [7]:
# merge home video data with preferred walking speed mat data 
# participants walk at preferred pace at home 

def merge_bw_hv(bw_df, hv_df, task, out_path):
    # filter zv to only include one task (bw drop columns, zv drop rows) 
    hv_task_df = hv_df[hv_df['task_pose_hv'].isin(task)]
    
    print('confirm all one task')
    print(pd.unique(hv_task_df['task_pose_hv']))

    print('total home videos') 
    print(len(hv_task_df))

    # filter bw ids dataset to only those included in video data set 
    hv_in_bw_df = bw_df[bw_df['bw_id'].isin(hv_task_df['id_video'])]
    hv_in_bw_df.to_csv(os.path.join(out_path, 'hv_id_in_bw_df.csv')) # save excel 

    print('total bw rows with id in video dataset') 
    print(len(hv_in_bw_df))

    # Track used rows from zv_in_bw_df
    used_indices = set()

    # Helper function to find the closest date
    def find_closest_date_unique(row, in_bw_df):    
        # Filter rows with the same 'bw_id' and not already used
        filtered_df = in_bw_df[(in_bw_df['bw_id'] == row['id_video'])] 
        # if no matching dates found 
        if filtered_df.empty:
            return None 
        # Find the closest date - visit date, some ppl don't have gait data at end of visit 
        closest_idx = (filtered_df['visit_date'] - row['visit_date_video']).abs().idxmin()
        return filtered_df.loc[closest_idx]

    # Apply the helper function row-wise
    closest_rows = hv_task_df.apply(
        lambda row: find_closest_date_unique(row, hv_in_bw_df), axis=1
    )
    
    # Convert the results into a DataFrame
   # closest_rows_df = pd.DataFrame(closest_rows.tolist(), index=hv_task_df.index)

    # Merge the original `zv_task_df` with `closest_rows_df`
    merged_bw_hv_df = hv_task_df.merge(closest_rows, left_index=True, right_index=True, suffixes=('', '_closest'))

    # add column for date diff 
    merged_bw_hv_df['bw_hv_date_diff_days'] = (merged_bw_hv_df['visit_date_video'] - merged_bw_hv_df['visit_date']).dt.days
    
    # check same ID for each row 
    print('mismatched home video vs brainwalk id')
    print(sum(merged_bw_hv_df['id_video'] != merged_bw_hv_df['bw_id']))
    
    return merged_bw_hv_df

In [8]:
# df input should be merged df - both video and bw data 

def print_video_counts(df):
    # number of zeno videos and participants included 
    print('total videos - df length: ' + 
          str(len(df))) 

    print('unique demographic_diagnosis in df: ' + 
         str(pd.unique(df['demographic_diagnosis'])))

    print('table value counts demographic_diagnosis') 
    print(df['demographic_diagnosis'].value_counts()) 

    print('------')

    print('unique id_video (participants) in df: ' + 
          str(len(pd.unique(df['id_video'])))) 

    print(f"num participants with demographic_diagnosis == HC: {df['id_video'][df['demographic_diagnosis'] == 'HC'].nunique()}")
    
    print(f"num participants with demographic_diagnosis == MS: {df['id_video'][df['demographic_diagnosis'] == 'MS'].nunique()}")

In [9]:
# merge race and ethnicity according to Bove lab patterns 

def merge_race_ethnicity(df): 
    df_2 = df
    df_2['race_ethnicity_clean'] = ''
    df_2 = df_2.copy()
    # White race and not hispanic or latino ethnicity = White Not Hispanic 
    df_2.loc[(df_2['clean_Race'] == 'White') & (df_2['clean_Ethnicity'] == 'Not Hispanic Or Latino'),
            'race_ethnicity_clean'] = 'White Non Hispanic' 

    # Exclude White Not Hispanic and ethnicity is Hispanic or Latino = 'Hispanic or Latino'
    df_2.loc[(df_2['race_ethnicity_clean'] != 'White Not Hispanic') & (df_2['clean_Ethnicity'] == 'Hispanic Or Latino'),
            'race_ethnicity_clean'] = 'Hispanic or Latino'

    # Of individuals that haven't been assignd race_ethnicity_clean, go off clean_Race column 
    # asian 
    df_2.loc[(df_2['race_ethnicity_clean'] == '') & (df_2['clean_Race'] == 'Asian'),
            'race_ethnicity_clean'] = 'Asian' 
    
   # Black Or African American 
    df_2.loc[(df_2['race_ethnicity_clean'] == '') & (df_2['clean_Race'] == 'Black Or African American'),
            'race_ethnicity_clean'] = 'Black Or African American'

    # if not yet assigned -> other, unknown, Declined 
    df_2.loc[df_2['race_ethnicity_clean'] == '',
            'race_ethnicity_clean'] = 'Other/Unknown/Declined'
    return df_2

In [10]:
def condense_ms_dx(df):
    df_2 = df
    df_2['ms_dx_condensed'] = ''
    df_2 = df_2.copy()
    
    # RRMS 
    df_2.loc[(df_2['bingoEHR_DX_MS DX'] == 'RRMS (Relapsing-remitting Multiple Sclerosis)') & 
             (df_2['demographic_diagnosis'] != 'HC'),
    'ms_dx_condensed'] = 'RRMS' 

    # Progressive MS  
    df_2.loc[(df_2['bingoEHR_DX_MS DX'] == 'PPMS (Primary-progressive Multiple Sclerosis)') | 
            (df_2['bingoEHR_DX_MS DX'] == 'SPMS (Secondary-progressive Multiple Sclerosis)') | 
            (df_2['bingoEHR_DX_MS DX'] == 'PRMS (Progressive-relapsing Multiple Sclerosis)') &  
            (df_2['demographic_diagnosis'] != 'HC'), 
    'ms_dx_condensed'] = 'Progressive MS' 

    # subtype not specified 
    df_2.loc[(df_2['bingoEHR_DX_MS DX'] == 'MS, Subtype Not Specified'),  'ms_dx_condensed'] = 'MS, Subtype Not Specified'

    # if still blank and demographic diagnosis column = MS -> 'MS, Subtype Not Specified'
    df_2.loc[(df_2['ms_dx_condensed'] == '') & (df_2['demographic_diagnosis'] == 'MS'),  
              'ms_dx_condensed'] = 'MS, Subtype Not Specified'

    # everyone else, Nan -> Likely healthy control  
    df_2.loc[(df_2['ms_dx_condensed'] == ''), 'ms_dx_condensed'] = None 

    # update 'demographic_diagnosis' column to match 'ms_dx_condensed'
#    df_2.loc[(df_2['ms_dx_condensed'] == 'RRMS') | 
#                 (df_2['ms_dx_condensed'] == 'Progressive MS') | 
#                 (df_2['ms_dx_condensed'] == 'MS, Subtype Not Specified'), 
#            'demographic_diagnosis'] = 'MS'

    return df_2

In [11]:
def categorize_multiple_people(df): 
    df_2 = df
    df_2['multiple_people_YN'] = ''
    df_2 = df_2.copy()

    # Yes + No ---------------------------------------------------------------
    # if multiple people in video
    df_2.loc[df_2['multiple_people'].isin(['yes_full_body_entire_video',
                                           'yes_full_body_part_of_video',
                                           'yes_part_body_entire_video',
                                           'yes_part_body_part_of_video']) , 'multiple_people_YN'] = 'Y'


    # if only one person in video 
    df_2.loc[df_2['multiple_people'] == 'no', 'multiple_people_YN'] = 'N' 

    # if other 
    df_2.loc[df_2['multiple_people'] == '', 'multiple_people_YN'] = 'UNK' 

    # three groups --------------------------------------------------
    df_2['multiple_people_3'] = ''
    df_2 = df_2.copy() 

    # 1 = full body at any time or part of body through entire video 
    df_2.loc[df_2['multiple_people'].isin(['yes_full_body_entire_video',
                                             'yes_full_body_part_of_video',
                                             'yes_part_body_entire_video']) , 'multiple_people_3'] = 'Group_1'

    # 2 - part of body for part of video 
    df_2.loc[df_2['multiple_people'].isin(['yes_part_body_part_of_video']) , 'multiple_people_3'] = 'Group_2'

    # 3 - no 
    df_2.loc[df_2['multiple_people'] == 'no', 'multiple_people_3'] = 'N' 

    # 
    df_2.loc[df_2['multiple_people'] == '', 'multiple_people_3'] = 'UNK' 

    return df_2
    

In [12]:
def assisitive_device_YN(df): 
    df_2 = df
    df_2['assistive_device_YN'] = ''
    df_2 = df_2.copy()

    # Yes + No ---------------------------------------------------------------
    df_2.loc[df_2['assistive_device'].isin(['single_cane', 'two_canes', 'walker']), 'assistive_device_YN'] = 'Y'

    df_2.loc[df_2['assistive_device'] == 'none', 'assistive_device_YN'] = 'N' 

    df_2.loc[df_2['assistive_device'] == '', 'assistive_device_YN'] = 'UNK'

    return df_2
    

## Load data and set output folders 

In [13]:
# output folders 
analysis_out_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis', 
                                analysis_version)

if not os.path.exists(analysis_out_path): 
    os.makedirs(analysis_out_path)

out_path = os.path.join(analysis_out_path, '000_merged_cleaned_data_temp') 

if not os.path.exists(out_path): 
    os.makedirs(out_path)

print(out_path)

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis\007\000_merged_cleaned_data_temp


In [14]:
# load data frames 

# zeno video metrics 
zv_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code',
                       'gait_bw_zeno_outputs_' + version, 
                       'gait_bw_zeno_outputs_' + version +'_pose_metrics_all.csv')

zv_df = pd.read_csv(zv_path, index_col = 0)

# home video metrics 
hv_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code',
                       'gait_bw_home_outputs_' + version, 
                       'gait_bw_home_outputs_' + version + '_pose_metrics_all.csv')
                       
hv_df = pd.read_csv(hv_path, index_col = 0)

# BW and zeno mat metrics 
# decrypted file - may need to decrypt again if says file doesn't exit 
# copied file saved in Megan Project folder in Brainwalk box
    # C:\Users\mmccu\Box\Brainwalk\Home Video Walking\Megan Project\bw_data_and_code
# if issues decrypting, try copying original file again and then decrypting 
bw_df = pd.read_excel(bw_path, 
                     index_col = None, 
                     usecols = ['bw_id', 'trialdate', 'visit_date', 'redcap_event_name', 'demoEHR_DiseaseDuration',
                                'clean_Sex', 'clean_Age', 'demographic_diagnosis','bingoEHR_DX_MS DX', 'bingoEHR_DX_MS DX', 'clean_Race',	
                                'clean_Ethnicity', 'clean_EDSS', 'bingoEHR_EDSS_Record_Date', 'demoEHR_EDSS_dateDiff', 'EDSS_same_before_after_MM', 'Riley_estimated_EDSS',
                                'clean_T25FW_Avg', 'msfcEHR_T25FW SPEED TRIAL 1 Record_Date',
                                'msfcEHR_T25FW SPEED TRIAL 1 vDate Diff', 'msfcEHR_T25FW SPEED TRIAL 2 vDate Diff',
                                'PWS_stridetimesecmean', 'PWS_stridetimeseccv','PWS_cadencestepsminmean','PWS_totaldsupportmean', 
                                'PWS_singlesupportmean','PWS_totaldsupportratiolr', 'PWS_singlesupportratiolr', 
                                'PWS_stridewidthcmmean','PWS_stridewidthcmsd', 'PWS_velocitycmsecmean', 
                                'FW_stridetimesecmean', 'FW_stridetimeseccv','FW_cadencestepsminmean','FW_totaldsupportmean', 
                                'FW_singlesupportmean','FW_totaldsupportratiolr', 'FW_singlesupportratiolr', 
                                'FW_stridewidthcmmean','FW_stridewidthcmsd', 'FW_velocitycmsecmean', 
                                'demoGait_dateDiff', 'demoEHR_DX_dateDiff', 'demoEHR_Vitals_dateDiff', 
                                'clean_SDMT_Correct', 'tc_Examinee_Education'])

bw_df.head()

,bw_id,trialdate,visit_date,demoGait_dateDiff,demoEHR_DiseaseDuration,bingoEHR_DX_MS DX,demographic_diagnosis,clean_Race,clean_Sex,clean_Ethnicity,...,msfcEHR_T25FW SPEED TRIAL 1 Record_Date,msfcEHR_T25FW SPEED TRIAL 1 vDate Diff,msfcEHR_T25FW SPEED TRIAL 2 vDate Diff,demoEHR_DX_dateDiff,bingoEHR_EDSS_Record_Date,demoEHR_EDSS_dateDiff,Riley_estimated_EDSS,EDSS_same_before_after_MM,demoEHR_Vitals_dateDiff,tc_Examinee_Education
0,BW-0146,NaT,2022-05-18,NaN,NaN,NaN,HC,White,Male,Not Hispanic Or Latino,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,BW-0146,2022-08-17,2022-08-17,0.0,NaN,NaN,HC,White,Male,Not Hispanic Or Latino,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,BW-0001,2022-08-26,2022-08-26,0.0,NaN,NaN,PD,White,Male,Not Hispanic Or Latino,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,BW-0002,2022-09-12,2022-09-12,0.0,0.0,RRMS (Relapsing-remitting Multiple Sclerosis),MS,Other,Male,Hispanic Or Latino,...,09/12/2022,0.0,0.0,97.0,2022-09-12,0.0,NaN,NaN,-0.454109,20.0
4,BW-0004,2022-09-19,2022-09-19,0.0,17.0,RRMS (Relapsing-remitting Multiple Sclerosis),MS,White,Female,Not Hispanic Or Latino,...,09/19/2022,0.0,0.0,6289.0,2022-09-19,0.0,NaN,NaN,-0.453264,16.0


## Format data 

In [15]:
# if version metric greater than 005 - drop extra support metrics 
# new calculation for support 
if int(version) >= 5: 
    zv_df = zv_df.loc[:, ~zv_df.columns.str.contains('foot')]
    hv_df = hv_df.loc[:, ~hv_df.columns.str.contains('foot')]

In [16]:
# rename zeno video variables 
# ad zv to all zeno volumn names 
zv_df = zv_df.add_suffix('_zv')

# add bw id and video date to zv df 
zv_df['id_video'] = zv_df['id_date_pose_zv'].str.extract(r'(BW-\d{4})')
zv_df['visit_date_video'] = zv_df['id_date_pose_zv'].str[8:]
zv_df['visit_date_video'] = pd.to_datetime(zv_df['visit_date_video'].str.replace('_', '-'), format='%Y-%m-%d')
zv_df.head()

,video_id_date_name_pose_zv,id_date_pose_zv,task_pose_zv,frames_per_second_pose_zv,total_video_duration_sec_pose_zv,delta_pix_h_rel_median_pose_zv,walking_segmets_n_pose_zv,walking_segments_duration_mean_pose_zv,walking_segments_duration_median_pose_zv,stride_time_mean_sec_pose_zv,...,frameDiff_to1a_hs1a_median_pose_zv,frameDiff_to1b_hs1b_median_pose_zv,frameDiff_to2_hs2_median_pose_zv,frameDiff_hs1a_to2_std_pose_zv,frameDiff_hs2_to1b_std_pose_zv,frameDiff_to1a_hs1a_std_pose_zv,frameDiff_to1b_hs1b_std_pose_zv,frameDiff_to2_hs2_std_pose_zv,id_video,visit_date_video
0,gait_vertical_PWS_1_BW-0002_2022_09_12,BW-0002\2022_09_12,gait_vertical_PWS_1,25,42.760000,0.15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW-0002,2022-09-12
1,gait_vertical_FW_1_BW-0002_2022_09_12,BW-0002\2022_09_12,gait_vertical_FW_1,25,28.960000,0.28,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW-0002,2022-09-12
2,gait_vertical_PWS_1_BW-0002_2023_09_12,BW-0002\2023_09_12,gait_vertical_PWS_1,30,46.633333,0.22,4.0,4.46,4.32,1.354,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW-0002,2023-09-12
3,gait_vertical_FW_1_BW-0002_2023_09_12,BW-0002\2023_09_12,gait_vertical_FW_1,30,36.500000,0.25,3.0,6.71,7.23,1.015,...,13.0,13.0,13.0,3.04,3.06,3.51,4.35,3.83,BW-0002,2023-09-12
4,gait_vertical_PWS_1_BW-0003_2022_10_24,BW-0003\2022_10_24,gait_vertical_PWS_1,30,26.033333,0.26,4.0,4.53,4.87,1.192,...,13.0,13.0,13.0,2.01,1.17,2.60,2.22,1.51,BW-0003,2022-10-24


In [17]:
# add hv to all home column names 
hv_df = hv_df.add_suffix('_hv')

# add bw id and video date to hv df 
hv_df['id_video'] = hv_df['id_date_pose_hv'].str.extract(r'(BW-\d{4})')
hv_df['visit_date_video'] = hv_df['id_date_pose_hv'].str[8:]
hv_df['visit_date_video'] = pd.to_datetime(hv_df['visit_date_video'], errors = 'coerce')
hv_df.head()

C:\Users\mmccu\AppData\Local\Temp\ipykernel_36456\2526253510.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hv_df['visit_date_video'] = pd.to_datetime(hv_df['visit_date_video'], errors = 'coerce')


,video_id_date_name_pose_hv,id_date_pose_hv,task_pose_hv,frames_per_second_pose_hv,total_video_duration_sec_pose_hv,delta_pix_h_rel_median_pose_hv,walking_segmets_n_pose_hv,walking_segments_duration_mean_pose_hv,walking_segments_duration_median_pose_hv,stride_time_mean_sec_pose_hv,...,frameDiff_to1b_hs1b_median_pose_hv,frameDiff_to2_hs2_median_pose_hv,frameDiff_hs1a_to2_std_pose_hv,frameDiff_hs2_to1b_std_pose_hv,frameDiff_to1a_hs1a_std_pose_hv,frameDiff_to1b_hs1b_std_pose_hv,frameDiff_to2_hs2_std_pose_hv,instruction_version_hv,id_video,visit_date_video
0,gait_vertical_left_BW-0018_10-24-23,BW-0018\10-24-23,gait_vertical_left,30,26.800000,0.22,6.0,2.88,2.92,1.182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BW-0018,2023-10-24
1,gait_vertical_right_BW-0018_10-24-23,BW-0018\10-24-23,gait_vertical_right,30,24.400000,0.31,6.0,3.15,3.20,1.185,...,10.0,10.0,2.97,2.30,2.92,1.91,1.79,1,BW-0018,2023-10-24
2,gait_vertical_left_BW-0023_05-05-23,BW-0023\05-05-23,gait_vertical_left,30,35.966667,0.43,5.0,4.43,4.47,1.054,...,11.0,10.5,4.69,1.78,2.84,2.81,2.50,1,BW-0023,2023-05-05
3,gait_vertical_right_BW-0023_05-05-23,BW-0023\05-05-23,gait_vertical_right,30,38.466667,0.39,6.0,4.79,4.77,1.091,...,13.0,13.0,3.29,2.09,2.37,2.58,2.47,1,BW-0023,2023-05-05
4,gait_vertical_left_BW-0023_10-23-23,BW-0023\10-23-23,gait_vertical_left,30,36.100000,0.30,3.0,4.27,3.93,1.008,...,9.0,9.0,2.19,1.09,3.05,1.67,1.44,1,BW-0023,2023-10-23


In [18]:
# add columns for ordinal EDSS severity and t25fw to bw_df
bw_df['edss_severity_num'], bw_df['edss_severity_cat'] = zip(*bw_df['clean_EDSS'].apply(categorize_edss))
bw_df['edss_severity_cat'] = pd.Categorical(bw_df['edss_severity_cat'], categories=["mild", "moderate", "severe"], ordered=True)

# Apply the function to create new columns to bw_df
bw_df['t25fw_group_num'], bw_df['t25fw_group_cat'] = zip(*bw_df['clean_T25FW_Avg'].apply(categorize_t25fw))
bw_df['t25fw_group_cat'] = pd.Categorical(bw_df['t25fw_group_cat'], categories=["under_6", "6_to_8", "over_8"], ordered=True)
bw_df.head()

,bw_id,trialdate,visit_date,demoGait_dateDiff,demoEHR_DiseaseDuration,bingoEHR_DX_MS DX,demographic_diagnosis,clean_Race,clean_Sex,clean_Ethnicity,...,bingoEHR_EDSS_Record_Date,demoEHR_EDSS_dateDiff,Riley_estimated_EDSS,EDSS_same_before_after_MM,demoEHR_Vitals_dateDiff,tc_Examinee_Education,edss_severity_num,edss_severity_cat,t25fw_group_num,t25fw_group_cat
0,BW-0146,NaT,2022-05-18,NaN,NaN,NaN,HC,White,Male,Not Hispanic Or Latino,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BW-0146,2022-08-17,2022-08-17,0.0,NaN,NaN,HC,White,Male,Not Hispanic Or Latino,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BW-0001,2022-08-26,2022-08-26,0.0,NaN,NaN,PD,White,Male,Not Hispanic Or Latino,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BW-0002,2022-09-12,2022-09-12,0.0,0.0,RRMS (Relapsing-remitting Multiple Sclerosis),MS,Other,Male,Hispanic Or Latino,...,2022-09-12,0.0,NaN,NaN,-0.454109,20.0,2.0,moderate,1.0,under_6
4,BW-0004,2022-09-19,2022-09-19,0.0,17.0,RRMS (Relapsing-remitting Multiple Sclerosis),MS,White,Female,Not Hispanic Or Latino,...,2022-09-19,0.0,NaN,NaN,-0.453264,16.0,2.0,moderate,1.0,under_6


In [19]:
# merge race and ethnicity columns in bw_df
bw_df = merge_race_ethnicity(bw_df)

In [20]:
print(bw_df['bingoEHR_DX_MS DX'].value_counts())
print(bw_df['demographic_diagnosis'].value_counts())
print('---- MS only -----')
print(bw_df.loc[bw_df['demographic_diagnosis'] == 'MS', 'bingoEHR_DX_MS DX'].value_counts())

# condense MS DX columns in bw_df
# add 'demographic_diagnosis' = MS if MS DX is MS diagnosis 
bw_df = condense_ms_dx(bw_df)

print('---- condensed ----') 
print(bw_df['ms_dx_condensed'].value_counts())
print(bw_df['demographic_diagnosis'].value_counts())

bingoEHR_DX_MS DX
RRMS (Relapsing-remitting Multiple Sclerosis)                       466
MS, Subtype Not Specified                                            33
SPMS (Secondary-progressive Multiple Sclerosis)                      29
PPMS (Primary-progressive Multiple Sclerosis)                        28
abnormal MRI                                                          3
CIS (Clinically Isolated Syndrome), with high risk MS (MRI, CSF)      1
PRMS (Progressive-relapsing Multiple Sclerosis)                       1
MOGAD                                                                 1
Name: count, dtype: int64
demographic_diagnosis
MS                                                                                 475
HC                                                                                  68
Mild TBI                                                                            38
PD                                                                                  19
bvFTD     

In [21]:
# save bw data with merged columns 
# save updated BW df 
bw_df.to_csv(os.path.join(out_path, 'mergedCols_' + bw_filename + '.csv'))
bw_df.head() 
bw_df.head()

,bw_id,trialdate,visit_date,demoGait_dateDiff,demoEHR_DiseaseDuration,bingoEHR_DX_MS DX,demographic_diagnosis,clean_Race,clean_Sex,clean_Ethnicity,...,Riley_estimated_EDSS,EDSS_same_before_after_MM,demoEHR_Vitals_dateDiff,tc_Examinee_Education,edss_severity_num,edss_severity_cat,t25fw_group_num,t25fw_group_cat,race_ethnicity_clean,ms_dx_condensed
0,BW-0146,NaT,2022-05-18,NaN,NaN,NaN,HC,White,Male,Not Hispanic Or Latino,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,White Non Hispanic,None
1,BW-0146,2022-08-17,2022-08-17,0.0,NaN,NaN,HC,White,Male,Not Hispanic Or Latino,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,White Non Hispanic,None
2,BW-0001,2022-08-26,2022-08-26,0.0,NaN,NaN,PD,White,Male,Not Hispanic Or Latino,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,White Non Hispanic,None
3,BW-0002,2022-09-12,2022-09-12,0.0,0.0,RRMS (Relapsing-remitting Multiple Sclerosis),MS,Other,Male,Hispanic Or Latino,...,NaN,NaN,-0.454109,20.0,2.0,moderate,1.0,under_6,Hispanic or Latino,RRMS
4,BW-0004,2022-09-19,2022-09-19,0.0,17.0,RRMS (Relapsing-remitting Multiple Sclerosis),MS,White,Female,Not Hispanic Or Latino,...,NaN,NaN,-0.453264,16.0,2.0,moderate,1.0,under_6,White Non Hispanic,RRMS


## Merge video data and brainwalk data 

In [22]:
# zeno videos - preferred walking speed 
bw_zv_pws_df = merge_bw_zv(bw_df, zv_df, 'gait_vertical_PWS_1', out_path) 

confirm all one task
['gait_vertical_PWS_1']
total zeno videos
305
total bw rows with id in video dataset
371
No matching id and daterow from video vs mat
BW-0036
2024-04-02 00:00:00
No matching id and daterow from video vs mat
BW-0121
2022-07-20 00:00:00
No matching id and daterow from video vs mat
BW-0322
2024-06-10 00:00:00
mismatched zeno video vs brainwalk id
0
mismatched zeno video vs brainwalk date
0


In [23]:
bw_zv_pws_df.head()

,video_id_date_name_pose_zv,id_date_pose_zv,task_pose_zv,frames_per_second_pose_zv,total_video_duration_sec_pose_zv,delta_pix_h_rel_median_pose_zv,walking_segmets_n_pose_zv,walking_segments_duration_mean_pose_zv,walking_segments_duration_median_pose_zv,stride_time_mean_sec_pose_zv,...,Riley_estimated_EDSS,EDSS_same_before_after_MM,demoEHR_Vitals_dateDiff,tc_Examinee_Education,edss_severity_num,edss_severity_cat,t25fw_group_num,t25fw_group_cat,race_ethnicity_clean,ms_dx_condensed
0,gait_vertical_PWS_1_BW-0002_2022_09_12,BW-0002\2022_09_12,gait_vertical_PWS_1,25,42.760000,0.15,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.454109,20.0,2.0,moderate,1.0,under_6,Hispanic or Latino,RRMS
1,gait_vertical_PWS_1_BW-0002_2023_09_12,BW-0002\2023_09_12,gait_vertical_PWS_1,30,46.633333,0.22,4.0,4.46,4.32,1.354,...,NaN,NaN,-11.391412,20.0,2.0,moderate,2.0,6_to_8,Hispanic or Latino,RRMS
2,gait_vertical_PWS_1_BW-0003_2022_10_24,BW-0003\2022_10_24,gait_vertical_PWS_1,30,26.033333,0.26,4.0,4.53,4.87,1.192,...,NaN,NaN,-7.471505,16.0,1.0,mild,1.0,under_6,Asian,"MS, Subtype Not Specified"
3,gait_vertical_PWS_1_BW-0004_2022_09_19,BW-0004\2022_09_19,gait_vertical_PWS_1,30,30.400000,0.37,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.453264,16.0,2.0,moderate,1.0,under_6,White Non Hispanic,RRMS
4,gait_vertical_PWS_1_BW-0006_2022_09_26,BW-0006\2022_09_26,gait_vertical_PWS_1,30,24.400000,0.36,2.0,4.63,4.63,1.148,...,NaN,NaN,-0.428669,18.0,2.0,moderate,1.0,under_6,White Non Hispanic,Progressive MS


In [24]:
print('----- print bw_zv_pws_df video counts ----')
print_video_counts(bw_zv_pws_df)

----- print bw_zv_pws_df video counts ----
total videos - df length: 302
unique demographic_diagnosis in df: ['MS' 'HC']
table value counts demographic_diagnosis
demographic_diagnosis
MS    258
HC     44
Name: count, dtype: int64
------
unique id_video (participants) in df: 206
num participants with demographic_diagnosis == HC: 39
num participants with demographic_diagnosis == MS: 167


In [25]:
# zeno videos - fast walking speed 
bw_zv_fw_df = merge_bw_zv(bw_df, zv_df, 'gait_vertical_FW_1', out_path)

confirm all one task
['gait_vertical_FW_1']
total zeno videos
312
total bw rows with id in video dataset
374
No matching id and daterow from video vs mat
BW-0036
2024-04-02 00:00:00
No matching id and daterow from video vs mat
BW-0322
2024-06-10 00:00:00
mismatched zeno video vs brainwalk id
0
mismatched zeno video vs brainwalk date
0


In [26]:
bw_zv_fw_df.head()

,video_id_date_name_pose_zv,id_date_pose_zv,task_pose_zv,frames_per_second_pose_zv,total_video_duration_sec_pose_zv,delta_pix_h_rel_median_pose_zv,walking_segmets_n_pose_zv,walking_segments_duration_mean_pose_zv,walking_segments_duration_median_pose_zv,stride_time_mean_sec_pose_zv,...,Riley_estimated_EDSS,EDSS_same_before_after_MM,demoEHR_Vitals_dateDiff,tc_Examinee_Education,edss_severity_num,edss_severity_cat,t25fw_group_num,t25fw_group_cat,race_ethnicity_clean,ms_dx_condensed
0,gait_vertical_FW_1_BW-0002_2022_09_12,BW-0002\2022_09_12,gait_vertical_FW_1,25,28.960000,0.28,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.454109,20.0,2.0,moderate,1.0,under_6,Hispanic or Latino,RRMS
1,gait_vertical_FW_1_BW-0002_2023_09_12,BW-0002\2023_09_12,gait_vertical_FW_1,30,36.500000,0.25,3.0,6.71,7.23,1.015,...,NaN,NaN,-11.391412,20.0,2.0,moderate,2.0,6_to_8,Hispanic or Latino,RRMS
2,gait_vertical_FW_1_BW-0003_2022_10_24,BW-0003\2022_10_24,gait_vertical_FW_1,30,28.233333,0.35,1.0,2.33,2.33,NaN,...,NaN,NaN,-7.471505,16.0,1.0,mild,1.0,under_6,Asian,"MS, Subtype Not Specified"
3,gait_vertical_FW_1_BW-0004_2022_09_19,BW-0004\2022_09_19,gait_vertical_FW_1,30,25.300000,0.36,2.0,3.15,3.15,1.433,...,NaN,NaN,-0.453264,16.0,2.0,moderate,1.0,under_6,White Non Hispanic,RRMS
4,gait_vertical_FW_1_BW-0006_2022_09_26,BW-0006\2022_09_26,gait_vertical_FW_1,30,24.966667,0.27,2.0,4.63,4.63,0.967,...,NaN,NaN,-0.428669,18.0,2.0,moderate,1.0,under_6,White Non Hispanic,Progressive MS


In [27]:
print('---- print bw_zv_fw_df video counts ----')
print_video_counts(bw_zv_fw_df)

---- print bw_zv_fw_df video counts ----
total videos - df length: 310
unique demographic_diagnosis in df: ['MS' 'HC']
table value counts demographic_diagnosis
demographic_diagnosis
MS    264
HC     46
Name: count, dtype: int64
------
unique id_video (participants) in df: 209
num participants with demographic_diagnosis == HC: 41
num participants with demographic_diagnosis == MS: 168


In [28]:
# home videos - merge home videos and BW 
print('----- right -------') 
bw_hv_r_pws_df = merge_bw_hv(bw_df, hv_df, ['gait_vertical_right', 'gait_vertical_right_2'], out_path)
print('----- left -------') 
bw_hv_l_pws_df = merge_bw_hv(bw_df, hv_df, ['gait_vertical_left', 'gait_vertical_left_2'], out_path)

----- right -------
confirm all one task
['gait_vertical_right' 'gait_vertical_right_2']
total home videos
43
total bw rows with id in video dataset
76
mismatched home video vs brainwalk id
0
----- left -------
confirm all one task
['gait_vertical_left' 'gait_vertical_left_2']
total home videos
41
total bw rows with id in video dataset
75
mismatched home video vs brainwalk id
0


In [29]:
# concatenate right and left 
bw_hv_pws_df = pd.concat([bw_hv_r_pws_df, bw_hv_l_pws_df], axis = 0).sort_index()
bw_hv_pws_df['edss_severity_cat'] = pd.Categorical(bw_hv_pws_df['edss_severity_cat'], categories=["mild", "moderate", "severe"], ordered=True)
print('---- print bw_hv_pws_df video counts ----')
print_video_counts(bw_hv_pws_df)

# save merged df  
bw_hv_pws_df.to_csv(os.path.join(out_path,  'hv_bw_merged_raw.csv'))

---- print bw_hv_pws_df video counts ----
total videos - df length: 84
unique demographic_diagnosis in df: ['MS']
table value counts demographic_diagnosis
demographic_diagnosis
MS    84
Name: count, dtype: int64
------
unique id_video (participants) in df: 38
num participants with demographic_diagnosis == HC: 0
num participants with demographic_diagnosis == MS: 38


## home: clean participants who sent multiple right or left videos at one timempoint 

In [30]:
# BW-0025: sent a right and left turn video incorrectly following instructions, then resnet (left_2 and right_2) correctly following instructions 
bw_hv_pws_df = bw_hv_pws_df[~((bw_hv_pws_df['bw_id'] == 'BW-0025') & (bw_hv_pws_df['walking_segmets_n_pose_hv'].isna()))]
bw_hv_pws_df.loc[bw_hv_pws_df['bw_id'] == 'BW-0025']

,video_id_date_name_pose_hv,id_date_pose_hv,task_pose_hv,frames_per_second_pose_hv,total_video_duration_sec_pose_hv,delta_pix_h_rel_median_pose_hv,walking_segmets_n_pose_hv,walking_segments_duration_mean_pose_hv,walking_segments_duration_median_pose_hv,stride_time_mean_sec_pose_hv,...,EDSS_same_before_after_MM,demoEHR_Vitals_dateDiff,tc_Examinee_Education,edss_severity_num,edss_severity_cat,t25fw_group_num,t25fw_group_cat,race_ethnicity_clean,ms_dx_condensed,bw_hv_date_diff_days
7,gait_vertical_left_2_BW-0025_11-28-2023,BW-0025\11-28-2023,gait_vertical_left_2,30,42.2,0.21,6.0,4.95,5.03,1.087,...,NaN,-0.472106,25.0,3.0,severe,1.0,under_6,White Non Hispanic,RRMS,-1
9,gait_vertical_right_2_BW-0025_11-28-2023,BW-0025\11-28-2023,gait_vertical_right_2,30,42.5,0.21,6.0,5.33,5.22,1.145,...,NaN,-0.472106,25.0,3.0,severe,1.0,under_6,White Non Hispanic,RRMS,-1


In [31]:
# clean participants who sent multiple right or left videos at one timempoint 
# Bw-0232: sent two right turn videos - one walks out of frame, one in frame 
bw_hv_pws_df = bw_hv_pws_df[~((bw_hv_pws_df['bw_id'] == 'BW-0232') & (bw_hv_pws_df['task_pose_hv'] == 'gait_vertical_right_2'))]
bw_hv_pws_df.loc[bw_hv_pws_df['bw_id'] == 'BW-0232']

,video_id_date_name_pose_hv,id_date_pose_hv,task_pose_hv,frames_per_second_pose_hv,total_video_duration_sec_pose_hv,delta_pix_h_rel_median_pose_hv,walking_segmets_n_pose_hv,walking_segments_duration_mean_pose_hv,walking_segments_duration_median_pose_hv,stride_time_mean_sec_pose_hv,...,EDSS_same_before_after_MM,demoEHR_Vitals_dateDiff,tc_Examinee_Education,edss_severity_num,edss_severity_cat,t25fw_group_num,t25fw_group_cat,race_ethnicity_clean,ms_dx_condensed,bw_hv_date_diff_days
40,gait_vertical_left_BW-0232_10-09-2023,BW-0232\10-09-2023,gait_vertical_left,10,22.6,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.522859,14.0,2.0,moderate,2.0,6_to_8,Asian,RRMS,5
41,gait_vertical_right_BW-0232_10-09-2023,BW-0232\10-09-2023,gait_vertical_right,10,28.7,0.09,3.0,3.67,3.7,1.036,...,NaN,-0.522859,14.0,2.0,moderate,2.0,6_to_8,Asian,RRMS,5


In [32]:
## Home - add columns for date diff between home video and EDSS 

In [33]:
bw_hv_pws_df['msfcEHR_T25FW SPEED TRIAL 1 Record_Date'][0]

'10/24/2023'

In [34]:
# format dates 
bw_hv_pws_df['msfcEHR_T25FW SPEED TRIAL 1 Record_Date'] = pd.to_datetime(bw_hv_pws_df['msfcEHR_T25FW SPEED TRIAL 1 Record_Date'], format='%m/%d/%Y')
bw_hv_pws_df['msfcEHR_T25FW SPEED TRIAL 1 Record_Date'].head()

0   2023-10-24
1   2023-10-24
2   2023-04-26
3   2023-04-26
4   2023-10-23
Name: msfcEHR_T25FW SPEED TRIAL 1 Record_Date, dtype: datetime64[ns]

In [35]:
# video and Zeno walk data 
bw_hv_pws_df['hv_zeno_walk_date_diff'] = (bw_hv_pws_df['visit_date_video'] - bw_hv_pws_df['trialdate']).dt.days

# video and EDSS 
bw_hv_pws_df['hv_edss_date_diff'] = (bw_hv_pws_df['visit_date_video'] - bw_hv_pws_df['bingoEHR_EDSS_Record_Date']).dt.days 

# video and T25FW 
bw_hv_pws_df['hv_t25fw_date_diff'] = (bw_hv_pws_df['visit_date_video'] - bw_hv_pws_df['msfcEHR_T25FW SPEED TRIAL 1 Record_Date']).dt.days

## Merge with July 2025 manual review
Megan and Jessica Li manually reviewed videos for videos with multiple people, assistive device, different lighting 

### Load Data 

In [36]:
# in-person
manual_zv_all_path = r"C:\Users\mmccu\Box\Brainwalk\Home Video Walking\Megan Project\Megan_Jessica Manual Video Review\review of in person videos.xlsx"
manual_zv_all_df = pd.read_excel(manual_zv_all_path, 
                                 index_col = None, 
                                 header = 2)

# drop duplicate rows before merge
manual_zv_all_df = manual_zv_all_df.drop(['visit_date',
                                          'demographic_diagnosis', 
                                          'clean_Race',
                                          'clean_Sex'],
                                         axis=1)

manual_zv_all_df.head()

C:\Users\mmccu\venv_home_video_analysis_2\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,id_video,visit_date_video,walking_speed,blinds,multiple_people,assistive_device,other_notes,remove_from_analysis,ad_pose,ad_world,...,notes_include_exclude,stride_time_high_error,cadence_high_error,stride_width_high_error,ssupport_high_error,dsupport_high_error,metric_notes,potential fixes,correct_person_pixel.1,notes
0,BW-0237,2024-10-03,PWS,fully_open,yes_part_body_part_of_video,none,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BW-0237,2024-10-03,FW,fully_open,yes_part_body_part_of_video,none,NaN,NaN,NaN,NaN,...,spins around the hips left to right a few time...,NaN,NaN,NaN,NaN,NaN,NaN,check right and left hip stay on same side (di...,yes_entire_video,NaN
2,BW-0274,2024-09-30,PWS,fully_open,yes_full_body_entire_video,none,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BW-0274,2024-09-30,FW,fully_open,yes_full_body_entire_video,none,NaN,NaN,NaN,NaN,...,maybe one or two frames where switches to coor...,NaN,NaN,NaN,NaN,NaN,"no support metrics calculated, rest seem reaso...",more filtering of hip data or loosen requireme...,yes_entire_video,NaN
4,BW-0125,2024-09-26,PWS,fully_open,no,none,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# home 
manual_hv_all_path = r"C:\Users\mmccu\Box\Brainwalk\Home Video Walking\Megan Project\Megan_Jessica Manual Video Review\review of home videos.xlsx"
manual_hv_all_df = pd.read_excel(manual_hv_all_path, 
                                 index_col = None, 
                                 header = 1)

# drop duplicate rows before merge
manual_hv_all_df = manual_hv_all_df.drop(['visit_date',
                                          'demographic_diagnosis', 
                                          'clean_Race',
                                          'clean_Sex'],
                                         axis=1) 

manual_hv_all_df.head()

C:\Users\mmccu\venv_home_video_analysis_2\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,id_video,visit_date_video,task_pose_hv,lighting,light_source,multiple_people,assistive_device,home_or_clinic,other_notes,video_speed_consistent,mp_spinning,reason_spinning,pixels_reasonable,pixels_notes,footfalls_in_frame_1,footfalls_in_frame_2,footfalls_in_frame_3,footfalls_in_frame_4,avg_footfalls,notes
0,BW-0018,2023-10-24,gait_vertical_left,well_lit,windows_visible,no,none,home,NaN,plots_faster,yes,window light,yes,tv reflection,4.0,3.0,4.0,2.0,3.25,NaN
1,BW-0018,2023-10-24,gait_vertical_right,well_lit,windows_visible,no,none,home,NaN,plots_faster,yes,window light,yes,NaN,3.0,3.0,4.0,2.0,3.00,plots seemed to go a little faster when body w...
2,BW-0023,2023-05-05,gait_vertical_left,well_lit,outdoors,no,none,home,dog present for most of video,plots_faster,no,NaN,yes,NaN,8.0,8.0,7.0,8.0,7.75,didn't track dog
3,BW-0023,2023-05-05,gait_vertical_right,well_lit,outdoors,no,none,home,dogs present in second half,plots_faster,no,NaN,yes,NaN,7.0,8.0,8.0,8.0,7.75,didn't track dog
4,BW-0023,2023-10-23,gait_vertical_left,well_lit,both_visible,no,none,clinic,NaN,plots_faster,no,NaN,yes,NaN,8.0,7.0,9.0,8.0,8.00,plots sped up when she was further away (harde...


### Merge 

In [38]:
# split FW and PWS 
manual_zv_pws_all_df = manual_zv_all_df.loc[manual_zv_all_df['walking_speed'] == 'PWS'] 
manual_zv_fw_all_df = manual_zv_all_df.loc[manual_zv_all_df['walking_speed'] == 'FW'] 

In [39]:
# merge manual review with PWS 
bw_zv_pws_df = bw_zv_pws_df.merge(manual_zv_pws_all_df, on = ['id_video', 'visit_date_video']) 

In [40]:
# merge manual review with FW
bw_zv_fw_df = bw_zv_fw_df.merge(manual_zv_fw_all_df, on = ['id_video', 'visit_date_video']) 

In [41]:
# merge manual review with home 
bw_hv_pws_df = bw_hv_pws_df.merge(manual_hv_all_df, on = ['id_video', 'visit_date_video', 'task_pose_hv']) 

### Categorize manual review columns and save new .csv with manual review notes 

In [42]:
# PWS 
bw_zv_pws_df = categorize_multiple_people(bw_zv_pws_df)
bw_zv_pws_df = assisitive_device_YN(bw_zv_pws_df)

bw_zv_pws_df.to_csv(os.path.join(out_path, 'zv_bw_merged_gait_vertical_PWS_1_wManualNotes.csv'))

In [43]:
# FW 
bw_zv_fw_df = categorize_multiple_people(bw_zv_fw_df)
bw_zv_fw_df = assisitive_device_YN(bw_zv_fw_df)

bw_zv_fw_df.to_csv(os.path.join(out_path, 'zv_bw_merged_gait_vertical_FW_1_wManualNotes.csv'))

In [44]:
# home
bw_hv_pws_df = categorize_multiple_people(bw_hv_pws_df)
bw_hv_pws_df = assisitive_device_YN(bw_hv_pws_df)

bw_hv_pws_df.to_csv(os.path.join(out_path, 'hv_bw_merged_wManualNotes.csv'))

## Drop rows with missing brainwalk data 
May still be missing some video metrics, but will filter within each relevant analysis step 
This step drops healthy controls 

In [46]:
# drop if missing any brainwalk data 

cols_to_check = ['demoEHR_DiseaseDuration', 'clean_Sex',
                 'race_ethnicity_clean', 'clean_Age',
                 'clean_EDSS', 'clean_T25FW_Avg',
                 'FW_cadencestepsminmean', 'FW_velocitycmsecmean',
                 'PWS_cadencestepsminmean', 'PWS_velocitycmsecmean']

cols_to_check_home = ['demoEHR_DiseaseDuration', 'clean_Sex',
                      'race_ethnicity_clean', 'clean_Age',
                      'clean_EDSS', 'clean_T25FW_Avg']

In [47]:
def remove_missing_data(df, columns_to_check):

    df_with_data = df.dropna(subset = columns_to_check)
    print(f"Videos in clean data frame: {len(df_with_data)}") 
    print(f"Unique participants in clean data frame: {df_with_data['bw_id'].nunique()}")

    #missing data 
    print('-----------') 
    df_ms = df.loc[df['demographic_diagnosis'] == 'MS']
    df_missing_data = df_ms[df_ms[columns_to_check].isna().any(axis=1)]
    print(f"Number videos with missing data: {len(df_missing_data)}")
    print(f"Unique participants missing data: {df_missing_data['bw_id'].nunique()}")
    if len(df_missing_data) > 0:
        print(df_missing_data[['bw_id', 'visit_date', 'demoEHR_DiseaseDuration', 'clean_Sex',
                               'race_ethnicity_clean', 'clean_Age',
                               'clean_EDSS', 'clean_T25FW_Avg',
                               'FW_cadencestepsminmean', 'FW_velocitycmsecmean',
                               'PWS_cadencestepsminmean', 'PWS_velocitycmsecmean']])

    return(df_with_data)

In [48]:
# PWS 
bw_zv_pws_clean_df = remove_missing_data(bw_zv_pws_df, cols_to_check)

Videos in clean data frame: 253
Unique participants in clean data frame: 163
-----------
Number videos with missing data: 5
Unique participants missing data: 5
       bw_id visit_date  demoEHR_DiseaseDuration clean_Sex  \
9    BW-0009 2022-10-05                      1.0    Female   
14   BW-0013 2022-10-03                     21.0    Female   
100  BW-0100 2023-02-23                      8.0      Male   
251  BW-0289 2024-02-14                      9.0      Male   
254  BW-0330 2024-06-14                      9.0    Female   

    race_ethnicity_clean  clean_Age  clean_EDSS  clean_T25FW_Avg  \
9     White Non Hispanic       32.5         NaN              3.5   
14    White Non Hispanic       53.8         6.0              NaN   
100   White Non Hispanic       35.2         NaN              7.2   
251   White Non Hispanic       74.1         4.5              8.8   
254                Asian       42.5         3.5              NaN   

     FW_cadencestepsminmean  FW_velocitycmsecmean  PWS_cad

In [49]:
# FW 
bw_zv_fw_clean_df = remove_missing_data(bw_zv_fw_df, cols_to_check)

Videos in clean data frame: 253
Unique participants in clean data frame: 163
-----------
Number videos with missing data: 11
Unique participants missing data: 11
       bw_id visit_date  demoEHR_DiseaseDuration clean_Sex  \
7    BW-0008 2022-09-27                     25.0      Male   
10   BW-0009 2022-10-05                      1.0    Female   
15   BW-0013 2022-10-03                     21.0    Female   
54   BW-0042 2024-04-04                      4.0    Female   
103  BW-0100 2023-02-23                      8.0      Male   
137  BW-0134 2023-09-21                     12.0    Female   
208  BW-0213 2024-07-31                      8.0    Female   
243  BW-0272 2024-06-12                      1.0      Male   
253  BW-0279 2024-02-14                     18.0    Female   
257  BW-0289 2024-02-14                      9.0      Male   
260  BW-0330 2024-06-14                      9.0    Female   

          race_ethnicity_clean  clean_Age  clean_EDSS  clean_T25FW_Avg  \
7           Hispani

In [50]:
# Home Videos 
bw_hv_pws_clean_df = remove_missing_data(bw_hv_pws_df, cols_to_check_home)

Videos in clean data frame: 81
Unique participants in clean data frame: 38
-----------
Number videos with missing data: 0
Unique participants missing data: 0


In [51]:
# BW data all - for demographics comparison in feasibility analysis 
#(peoplewith demographic and MS info, but not necessarily gait data) 
bw_clean_df = remove_missing_data(bw_df, cols_to_check_home)

Videos in clean data frame: 397
Unique participants in clean data frame: 245
-----------
Number videos with missing data: 80
Unique participants missing data: 77
       bw_id visit_date  demoEHR_DiseaseDuration clean_Sex  \
9    BW-0013 2022-10-03                     21.0    Female   
11   BW-0009 2022-10-05                      1.0    Female   
94   BW-0100 2023-02-23                      8.0      Male   
193  BW-0186 2023-07-13                     13.0      Male   
211  BW-0206 2023-08-01                      9.0      Male   
..       ...        ...                      ...       ...   
691  BW-0484 2025-04-01                     24.0    Female   
698  BW-0481 2025-04-08                      NaN      Male   
699  BW-0508 2025-04-08                      NaN      Male   
704  BW-0504 2025-04-16                     11.0    Female   
705  BW-0503 2025-04-16                     20.0    Female   

       race_ethnicity_clean  clean_Age  clean_EDSS  clean_T25FW_Avg  \
9        White Non His

## Check rows with data out of range 
T25FW: Check if within 90 days of BW visit 

EDSS: Check if within 365 days of BW visit OR stays stable before and after visits 

Home Videos: video date vs EDSS and vs T25FW? 

In [52]:
def remove_out_of_range(df):
    # T25FW
    df_in_range = df.loc[(df['msfcEHR_T25FW SPEED TRIAL 1 vDate Diff'] <= 365) & (df['msfcEHR_T25FW SPEED TRIAL 1 vDate Diff'] >= -365)]
    
    # EDSS
    df_in_range = df_in_range.loc[(df_in_range['demoEHR_EDSS_dateDiff'] <= 365) & (df_in_range['demoEHR_EDSS_dateDiff'] >= -365) | 
    (df_in_range['EDSS_same_before_after_MM'] == "Y")] 

    # print numbers in clean data set 
    print('Clean dataset in range - used in analysis')
    print(f"Videos in range: {len(df_in_range)}") 
    print(f"Unique participants in range: {df_in_range['bw_id'].nunique()}")

    print('------------------------') 
    print('T25FW or EDSS out of range') 
    out_of_range_df = df.loc[(df['msfcEHR_T25FW SPEED TRIAL 1 vDate Diff'] > 365) | (df['msfcEHR_T25FW SPEED TRIAL 1 vDate Diff'] < -365) | 
    (df['demoEHR_EDSS_dateDiff'] > 365) | (df['demoEHR_EDSS_dateDiff'] < -365) ]
    out_of_range_df = out_of_range_df.loc[out_of_range_df['EDSS_same_before_after_MM'] != 'Y'] # people with same EDSS were included 
    print(f"Number videos with EDSS or T25FW out of range: {len(out_of_range_df)}")
    print(f"Unique participants with EDSS or T25FW out of range: {out_of_range_df['bw_id'].nunique()}")
    if len(out_of_range_df) > 0:
        print(out_of_range_df[['bw_id', 'visit_date', 'clean_T25FW_Avg', 'msfcEHR_T25FW SPEED TRIAL 1 vDate Diff', 'clean_EDSS', 'demoEHR_EDSS_dateDiff', 'EDSS_same_before_after_MM']])

    return(df_in_range) 

In [53]:
# PWS 
bw_zv_pws_in_range_df = remove_out_of_range(bw_zv_pws_clean_df)
bw_zv_pws_in_range_df.to_csv(os.path.join(out_path, 'zv_bw_merged_gait_vertical_PWS_1_clean.csv'))

Clean dataset in range - used in analysis
Videos in range: 232
Unique participants in range: 153
------------------------
T25FW or EDSS out of range
Number videos with EDSS or T25FW out of range: 21
Unique participants with EDSS or T25FW out of range: 19
       bw_id visit_date  clean_T25FW_Avg  \
54   BW-0044 2022-11-08              4.9   
58   BW-0045 2022-11-10              6.6   
63   BW-0049 2024-04-22              3.7   
69   BW-0054 2022-11-14              5.7   
80   BW-0064 2022-11-21              6.1   
81   BW-0064 2024-07-15              6.1   
101  BW-0100 2024-04-17              7.2   
142  BW-0148 2023-05-01              4.2   
143  BW-0148 2024-08-22              4.2   
147  BW-0153 2023-05-10              4.0   
154  BW-0161 2023-05-26              5.2   
164  BW-0166 2024-06-17              4.2   
168  BW-0168 2023-12-06              4.1   
176  BW-0177 2023-06-27             10.8   
179  BW-0180 2023-06-28              5.0   
240  BW-0273 2024-08-29              5.4 

In [54]:
# FW 
bw_zv_fw_in_range_df = remove_out_of_range(bw_zv_fw_clean_df)
bw_zv_fw_in_range_df.to_csv(os.path.join(out_path, 'zv_bw_merged_gait_vertical_FW_1_clean.csv'))

Clean dataset in range - used in analysis
Videos in range: 232
Unique participants in range: 153
------------------------
T25FW or EDSS out of range
Number videos with EDSS or T25FW out of range: 21
Unique participants with EDSS or T25FW out of range: 19
       bw_id visit_date  clean_T25FW_Avg  \
56   BW-0044 2022-11-08              4.9   
60   BW-0045 2022-11-10              6.6   
65   BW-0049 2024-04-22              3.7   
71   BW-0054 2022-11-14              5.7   
82   BW-0064 2022-11-21              6.1   
83   BW-0064 2024-07-15              6.1   
104  BW-0100 2024-04-17              7.2   
145  BW-0148 2023-05-01              4.2   
146  BW-0148 2024-08-22              4.2   
150  BW-0153 2023-05-10              4.0   
157  BW-0161 2023-05-26              5.2   
167  BW-0166 2024-06-17              4.2   
171  BW-0168 2023-12-06              4.1   
179  BW-0177 2023-06-27             10.8   
182  BW-0180 2023-06-28              5.0   
245  BW-0273 2024-08-29              5.4 

In [55]:
# Home Videos  
bw_hv_pws_in_range_df = remove_out_of_range(bw_hv_pws_clean_df)
bw_hv_pws_in_range_df.to_csv(os.path.join(out_path, 'hv_bw_merged_clean.csv'))

Clean dataset in range - used in analysis
Videos in range: 81
Unique participants in range: 38
------------------------
T25FW or EDSS out of range
Number videos with EDSS or T25FW out of range: 0
Unique participants with EDSS or T25FW out of range: 0


In [56]:
# BW All Data -  for demographics comparison in feasibility analysis 
bw_in_range_df = remove_out_of_range(bw_clean_df)
# save updated BW df 
bw_in_range_df.to_csv(os.path.join(out_path, 'clean_mergedCols_' + bw_filename + '.csv'))
#bw_in_range_df.head() 

Clean dataset in range - used in analysis
Videos in range: 312
Unique participants in range: 176
------------------------
T25FW or EDSS out of range
Number videos with EDSS or T25FW out of range: 49
Unique participants with EDSS or T25FW out of range: 46
       bw_id visit_date  clean_T25FW_Avg  \
44   BW-0045 2022-11-10              6.6   
45   BW-0054 2022-11-14              5.7   
58   BW-0064 2022-11-21              6.1   
147  BW-0148 2023-05-01              4.2   
166  BW-0044 2022-11-08              4.9   
240  BW-0049 2024-04-22              3.7   
278  BW-0064 2024-07-15              6.1   
302  BW-0080 2024-11-12              4.6   
349  BW-0148 2024-08-22              4.2   
354  BW-0148 2025-03-10              4.4   
370  BW-0100 2024-04-17              7.2   
376  BW-0153 2023-05-10              4.0   
391  BW-0161 2023-05-26              5.2   
401  BW-0166 2024-06-17              4.2   
419  BW-0168 2023-12-06              4.1   
442  BW-0177 2023-06-27             10.8 